In [1]:
import pandas as pd
import numpy as np
import xarray as xr

import os
import re

import tqdm

% matplotlib inline

In [2]:
# ! pip install fuzzywuzzy[speedup]

In [3]:
import fuzzywuzzy as fw

In [4]:
from fuzzywuzzy import process, fuzz

In [5]:
import geopandas as gpd

In [6]:
gpd.__version__

'1.0.0.dev+141.ge925363'

Make sure your version can handle vectorized operations. If in doubt:

```conda install -c conda-forge/label/dev geopandas```

# read in data

In [7]:
! head -n 3 DataVizDisasterSummariesFV12.19.2016.csv

Disaster Number	Year	Region	Region Number	State Abbreviation	State	County	Declaration Date	Disaster Type	Incident Type	Title	Incident Begin Date	Incident End Date	Disaster Close Out Date
3	1953	Region VI	6	LA	Louisiana	Statewide	5/29/53	DR	Flood	FLOOD	5/29/53	5/29/53	2/1/60
1	1953	Region IV	4	GA	Georgia	Statewide	5/2/53	DR	Tornado	TORNADO	5/2/53	5/2/53	6/1/54


In [8]:
df = pd.read_csv('DataVizDisasterSummariesFV12.19.2016.csv', sep='\t')

In [9]:
df.head()

,Disaster Number,Year,Region,Region Number,State Abbreviation,State,County,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,Disaster Close Out Date
0,3,1953,Region VI,6,LA,Louisiana,Statewide,5/29/53,DR,Flood,FLOOD,5/29/53,5/29/53,2/1/60
1,1,1953,Region IV,4,GA,Georgia,Statewide,5/2/53,DR,Tornado,TORNADO,5/2/53,5/2/53,6/1/54
2,4,1953,Region V,5,MI,Michigan,Statewide,6/2/53,DR,Tornado,TORNADO,6/2/53,6/2/53,2/1/56
3,15,1954,Region IX,9,CA,California,Statewide,2/5/54,DR,Flood,FLOOD & EROSION,2/5/54,2/5/54,9/1/57
4,25,1954,Region I,1,CT,Connecticut,Statewide,9/17/54,DR,Hurricane,HURRICANES,9/17/54,9/17/54,1/1/57


In [10]:
len(df), len(df.drop_duplicates())

(46027, 45991)

In [11]:
df = df.drop_duplicates()

In [12]:
len(df)

45991

## Assign territories to disaster declarations

### Read in US census FIPS codes

In [13]:
fips = pd.read_csv('all-geocodes-v2016.csv', skiprows=4)

In [14]:
fips.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,0,0,0,0,United States
1,40,1,0,0,0,0,Alabama
2,50,1,1,0,0,0,Autauga County
3,50,1,3,0,0,0,Baldwin County
4,50,1,5,0,0,0,Barbour County


In [15]:
fips.columns = ['summary_level', 'state_code', 'county_code', 'blah1', 'blah2', 'blah3', 'area_name']

In [16]:
valid_fips_suffixes = (
    fips[(fips.summary_level == 50) & (fips.area_name != 'District of Columbia')]
    .area_name
    .apply(lambda x: x.split(' ')[-1])
    .unique())

valid_fips_suffixes

array(['County', 'Borough', 'Area', 'Municipality', 'Parish', 'city',
       'City', 'Municipio'], dtype=object)

In [17]:
county_fips_table = (
    fips
    .loc[fips.summary_level == 50, ['state_code', 'county_code', 'area_name']]
    .set_index(['state_code', 'county_code']))

In [18]:
county_fips_table.head()

area_name
state_code county_code                
1          1            Autauga County
           3            Baldwin County
           5            Barbour County
           7               Bibb County
           9             Blount County

In [19]:
state_fips = pd.read_csv('state_fips_codes.csv')

In [20]:
state_fips.head()

,name,fips_state_numeric_code,official_usps_code
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA


In [21]:
state_fips_by_abbrev = state_fips.set_index('official_usps_code').fips_state_numeric_code

In [22]:
state_fips_by_abbrev.head()

official_usps_code
AL    1
AK    2
AZ    4
AR    5
CA    6
Name: fips_state_numeric_code, dtype: int64

### Examine unique county names

In [23]:
unique_counties = df.County.unique()
unique_counties

array(['Statewide', 'Del Norte County', 'Nevada County', ...,
       'Pribilof Island Regional Educational Attendance Area',
       'Chugach Regional Educational Attendance Area',
       'Aleutians West Census Area'], dtype=object)

In [24]:
statewides = ['Statewide', 'District of Columbia']

In [25]:
# find unique region names that are not a state or DC
not_states = unique_counties[~np.isin(unique_counties, statewides)]

# mask counties that have suffixes in the fips code suffix list
actual_county_mask = np.isin(
    [c.split(' ')[-1].lower() for c in not_states],
    list(map(lambda x: x.lower(), valid_fips_suffixes)))

# create a list of validated county names
actual_counties = not_states[actual_county_mask]

# create a list of invalid county names
not_actual_counties = not_states[~actual_county_mask]

In [26]:
not_actual_counties

array(['Western District', "Manu'a District", 'Eastern District',
       'Koror State', 'St. Thomas Island', 'St. John Island',
       'St. Croix Island', 'Palau Island State', 'Guam', 'Kili', 'Ujae',
       'Namu', 'Ailinglaplap', 'Ebeye', 'Enewetak', 'Ebon', 'Jaluit',
       'Kosrae', 'Truk', 'Yap', 'Majuro', 'Ponape', 'Kwajalein',
       'Pohnpei', 'Arno', 'Aur', 'Lae', 'Lib', 'Mili', 'Utrik', 'Ujelang',
       'Ailuk', 'Wotje', 'Wotho', 'Bikini', 'Likiep', 'Mejit', 'Maloelap',
       'Namorik', 'Jabat', 'Chuuk'], dtype=object)

In [27]:
df.loc[np.isin(df.County, not_actual_counties), ['State Abbreviation', 'State', 'County']].drop_duplicates()

,State Abbreviation,State,County
424,AS,American Samoa,Western District
646,AS,American Samoa,Manu'a District
684,AS,American Samoa,Eastern District
828,PW,Republic of Palau,Koror State
2084,VI,Virgin Islands,St. Thomas Island
2122,VI,Virgin Islands,St. John Island
3898,VI,Virgin Islands,St. Croix Island
4596,PW,Republic of Palau,Palau Island State
4695,GU,Guam,Guam
5075,MH,Marshall Islands,Kili


## Create fips column & assign values

### Create column

In [28]:
df['fips'] = np.nan

### Assign statewide values

In [29]:
df.loc[np.isin(df.County, statewides), 'fips'] = (
    state_fips_by_abbrev.reindex(df.loc[np.isin(df.County, statewides), 'State Abbreviation']).values)

In [30]:
df[['State Abbreviation', 'State', 'fips']].dropna(how='any').drop_duplicates()

,State Abbreviation,State,fips
0,LA,Louisiana,22.0
1,GA,Georgia,13.0
2,MI,Michigan,26.0
3,CA,California,6.0
4,CT,Connecticut,9.0
5,OK,Oklahoma,40.0
6,ID,Idaho,16.0
7,NY,New York,36.0
9,MO,Missouri,29.0
10,TX,Texas,48.0


that looks good!

### Assign county FIPS codes

In [31]:
counties = (
    df
    .loc[~np.isin(df.County, statewides), ['State Abbreviation', 'County']]
    .drop_duplicates())

In [32]:
counties.head()

,State Abbreviation,County
68,CA,Del Norte County
69,CA,Nevada County
70,OR,Hood River County
71,OR,Marion County
72,OR,Clatsop County


In [33]:
len(counties)

3268

In [34]:
sample_county = counties.iloc[0]
sample_county

State Abbreviation                  CA
County                Del Norte County
Name: 68, dtype: object

In [35]:
def find_county_by_state_abbrev_and_name(state_fips, abbrev, name):
    try:
        state_counties = county_fips_table.xs(state_fips, level='state_code')
    except KeyError:
        return np.nan

    matched = np.isin(state_counties.area_name.values, name)

    if matched.any():
        candidates = state_counties.index.values[matched]
        if len(candidates) > 1:
            raise ValueError('more than one county matched for "{}, {}": "{}"'.format(
                name,
                abbrev,
                candidates))

        return candidates[0]

    else:
        return np.nan

def find_county_from_county_series(ser):
    return find_county_by_state_abbrev_and_name(ser['state_fips'], ser['State Abbreviation'], ser.County)

In [36]:
df['state_fips'] = state_fips_by_abbrev.reindex(df['State Abbreviation']).values
df.head()

,Disaster Number,Year,Region,Region Number,State Abbreviation,State,County,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,Disaster Close Out Date,fips,state_fips
0,3,1953,Region VI,6,LA,Louisiana,Statewide,5/29/53,DR,Flood,FLOOD,5/29/53,5/29/53,2/1/60,22.0,22
1,1,1953,Region IV,4,GA,Georgia,Statewide,5/2/53,DR,Tornado,TORNADO,5/2/53,5/2/53,6/1/54,13.0,13
2,4,1953,Region V,5,MI,Michigan,Statewide,6/2/53,DR,Tornado,TORNADO,6/2/53,6/2/53,2/1/56,26.0,26
3,15,1954,Region IX,9,CA,California,Statewide,2/5/54,DR,Flood,FLOOD & EROSION,2/5/54,2/5/54,9/1/57,6.0,6
4,25,1954,Region I,1,CT,Connecticut,Statewide,9/17/54,DR,Hurricane,HURRICANES,9/17/54,9/17/54,1/1/57,9.0,9


In [37]:
df['county_fips'] = df.apply(find_county_from_county_series, axis=1)

In [38]:
df.loc[np.isin(df.County, statewides), 'fips'] = df.state_fips * 1000
df.loc[~np.isin(df.County, statewides), 'fips'] = df.state_fips * 1000 + df.county_fips

In [39]:
df.head()

,Disaster Number,Year,Region,Region Number,State Abbreviation,State,County,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,Disaster Close Out Date,fips,state_fips,county_fips
0,3,1953,Region VI,6,LA,Louisiana,Statewide,5/29/53,DR,Flood,FLOOD,5/29/53,5/29/53,2/1/60,22000.0,22,NaN
1,1,1953,Region IV,4,GA,Georgia,Statewide,5/2/53,DR,Tornado,TORNADO,5/2/53,5/2/53,6/1/54,13000.0,13,NaN
2,4,1953,Region V,5,MI,Michigan,Statewide,6/2/53,DR,Tornado,TORNADO,6/2/53,6/2/53,2/1/56,26000.0,26,NaN
3,15,1954,Region IX,9,CA,California,Statewide,2/5/54,DR,Flood,FLOOD & EROSION,2/5/54,2/5/54,9/1/57,6000.0,6,NaN
4,25,1954,Region I,1,CT,Connecticut,Statewide,9/17/54,DR,Hurricane,HURRICANES,9/17/54,9/17/54,1/1/57,9000.0,9,NaN


In [40]:
df.loc[df.fips.isnull(), ['State', 'State Abbreviation', 'County', 'fips', 'state_fips', 'county_fips']].drop_duplicates()

,State,State Abbreviation,County,fips,state_fips,county_fips
424,American Samoa,AS,Western District,NaN,60,NaN
646,American Samoa,AS,Manu'a District,NaN,60,NaN
684,American Samoa,AS,Eastern District,NaN,60,NaN
828,Republic of Palau,PW,Koror State,NaN,70,NaN
1718,Puerto Rico,PR,Bayamon Municipio,NaN,72,NaN
1899,Puerto Rico,PR,Juana Diaz Municipio,NaN,72,NaN
1901,Puerto Rico,PR,San Sebastian Municipio,NaN,72,NaN
1936,Puerto Rico,PR,Guanica Municipio,NaN,72,NaN
1937,Puerto Rico,PR,Penuelas Municipio,NaN,72,NaN
1970,Puerto Rico,PR,Catano Municipio,NaN,72,NaN


# Attempt to match based on spelling variations

In [41]:
county_fips_table.index.get_level_values('state_code').unique()

Int64Index([ 1,  2,  4,  5,  6,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20,
            21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
            38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56,
            72],
           dtype='int64', name='state_code')

In [42]:
missing_counties = df.loc[df.fips.isnull(), ['State Abbreviation', 'state_fips', 'County']].drop_duplicates()

for i in tqdm.tqdm_notebook(range(len(missing_counties))):
    state_abbrev, state_fips, county_name = missing_counties.iloc[i]

    try:
        candidate_names = county_fips_table.xs(int(state_fips), level='state_code').area_name
    except KeyError:
        continue

    candidate, match_share, cfips = process.extractOne(
        county_name,
        candidate_names)
    
    if match_share > 90:
        df.loc[(df.state_fips == state_fips) & (df.County == county_name), 'county_fips'] = cfips
        print('matched "{}, {}" to {} ({:3.1%} match)'.format(county_name, state_abbrev, candidate, match_share/100))

matched "Bayamon Municipio, PR" to Bayamón Municipio (97.0% match)
matched "Juana Diaz Municipio, PR" to Juana Díaz Municipio (97.0% match)
matched "San Sebastian Municipio, PR" to San Sebastián Municipio (98.0% match)
matched "Guanica Municipio, PR" to Guánica Municipio (97.0% match)
matched "Penuelas Municipio, PR" to Peñuelas Municipio (97.0% match)
matched "Catano Municipio, PR" to Cataño Municipio (97.0% match)
matched "Rio Grande Municipio, PR" to Río Grande Municipio (97.0% match)
matched "Loiza Municipio, PR" to Loíza Municipio (97.0% match)
matched "Manati Municipio, PR" to Manatí Municipio (97.0% match)
matched "Comerio Municipio, PR" to Comerío Municipio (97.0% match)
matched "Dona Ana County, NM" to Doña Ana County (97.0% match)
matched "La Salle Parish, LA" to LaSalle Parish (97.0% match)
matched "Anasco Municipio, PR" to Añasco Municipio (97.0% match)
matched "Canovanas Municipio, PR" to Canóvanas Municipio (97.0% match)
matched "Las Marias Municipio, PR" to Las Marías Mu

In [43]:
df.loc[np.isin(df.County, statewides), 'fips'] = df.state_fips * 1000
df.loc[~np.isin(df.County, statewides), 'fips'] = df.state_fips * 1000 + df.county_fips

### Assign state fips values to overseas territories with no counties

In [44]:
df.loc[np.isin(df['State Abbreviation'], ['AS', 'PW', 'VI', 'MP', 'GU', 'MH', 'FM']), 'state_fips'] = (
    state_fips_by_abbrev.reindex(
        df.loc[np.isin(df['State Abbreviation'], ['AS', 'PW', 'VI', 'MP', 'GU', 'MH', 'FM']), 'State Abbreviation'])
    .values)

In [45]:
state_fips = (np.isin(df.County, statewides) | np.isin(df['State Abbreviation'], ['AS', 'PW', 'VI', 'MP', 'GU', 'MH', 'FM']))
df.loc[state_fips, 'fips'] = df.state_fips * 1000
df.loc[~state_fips, 'fips'] = df.state_fips * 1000 + df.county_fips

### Manually assign Bedford City, VA

It was later incorporated into Bedford County, VA

In [46]:
process.extractOne(
    "Bedford city",
    county_fips_table.xs(51, level='state_code').area_name,
    scorer=fuzz.token_sort_ratio)

('Bedford County', 85, 19)

In [47]:
df.loc[((df.State == 'Virginia') & (df.County == 'Bedford city')), 'state_fips'] = 51
df.loc[((df.State == 'Virginia') & (df.County == 'Bedford city')), 'fips'] = 51019

### Handle AK Educational Attendence Areas

In [48]:
df.loc[df.fips.isnull(), ['State', 'State Abbreviation', 'County', 'fips', 'state_fips', 'county_fips']].drop_duplicates()

,State,State Abbreviation,County,fips,state_fips,county_fips
20089,Alaska,AK,Yukon Koyukuk Regional Educational Attendance ...,NaN,2,NaN
20224,Alaska,AK,Alaska Gateway Regional Educational Attendance...,NaN,2,NaN
20227,Alaska,AK,Yukon Flats Regional Educational Attendance Area,NaN,2,NaN
20880,Alaska,AK,Lower Yukon Regional Educational Attendance Area,NaN,2,NaN
22134,Alaska,AK,Lower Kuskokwim Regional Educational Attendanc...,NaN,2,NaN
36854,Alaska,AK,Kuspuk Regional Educational Attendance Area,NaN,2,NaN
37829,Alaska,AK,Yupiit Regional Educational Attendance Area,NaN,2,NaN
41657,Alaska,AK,Bering Strait Regional Educational Attendance ...,NaN,2,NaN
43017,Alaska,AK,Copper River Regional Educational Attendance Area,NaN,2,NaN
43423,Alaska,AK,Southwest Region Regional Educational Attendan...,NaN,2,NaN


In [49]:
ak_districts = gpd.read_file('cb_2017_02_unsd_500k/cb_2017_02_unsd_500k.shp')
ak_districts.head()

I am densified (5 elements)
I am densified (5 elements)


,STATEFP,UNSDLEA,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,02,00004,9700000US0200004,0200004,Yupiit School District,00,33902542,9284194,"(POLYGON ((-160.984191 61.11598799999999, -160..."
1,02,00270,9700000US0200270,0200270,Haines Borough School District,00,6233111973,1054089659,"(POLYGON ((-135.085227 58.266554, -135.076908 ..."
2,02,00510,9700000US0200510,0200510,Matanuska-Susitna Borough School District,00,63760578688,1704714297,"POLYGON ((-153.001506 62.727441, -152.997647 6..."
3,02,00540,9700000US0200540,0200540,Nenana City School District,00,14885242,763703,"POLYGON ((-149.134904 64.527525, -149.128806 6..."
4,02,00715,9700000US0200715,0200715,Tanana City School District,00,27634833,11209197,"POLYGON ((-152.154481 65.19625499999999, -152...."


In [50]:
ak_districts.NAME.unique()

array(['Yupiit School District', 'Haines Borough School District',
       'Matanuska-Susitna Borough School District',
       'Nenana City School District', 'Tanana City School District',
       'Hydaburg City School District',
       'Southeast Island School District',
       'North Slope Borough School District',
       'Alaska Gateway School District', 'Kashunamiut School District',
       'Petersburg Borough School District',
       'Delta/Greely School District', 'Galena City School District',
       'Yukon-Koyukuk School District', 'Yukon Flats School District',
       'St. Marys City School District',
       'Fairbanks North Star Borough School District',
       'Valdez City School District', 'Bering Strait School District',
       'Pelican City School District', 'Yakutat School District',
       'Unalaska City School District', 'Anchorage School District',
       'Dillingham City School District',
       'Southwest Region School District',
       'Ketchikan Gateway Borough Scho

In [51]:
df['school_district'] = np.nan

In [52]:
ak_district_lookup = ak_districts.set_index('NAME').GEOID.astype(int)

In [53]:
ak_district_lookup.reindex(
    df.loc[(df.fips.isnull()), 'County']
    .str.replace('Regional Educational Attendance Area', 'School District')
    .str.replace('Yukon Koyukuk', 'Yukon-Koyukuk')
    .str.replace('Pribilof Island', 'Pribilof'))

County
Yukon-Koyukuk School District       200862
Alaska Gateway School District      200050
Yukon Flats School District         200775
Lower Yukon School District         200003
Lower Yukon School District         200003
Yukon Flats School District         200775
Lower Yukon School District         200003
Lower Kuskokwim School District     200001
Yukon-Koyukuk School District       200862
Lower Yukon School District         200003
Yukon-Koyukuk School District       200862
Yukon Flats School District         200775
Kuspuk School District              200760
Yukon-Koyukuk School District       200862
Lower Kuskokwim School District     200001
Yupiit School District              200004
Lower Yukon School District         200003
Kuspuk School District              200760
Bering Strait School District       200020
Yukon-Koyukuk School District       200862
Bering Strait School District       200020
Lower Kuskokwim School District     200001
Alaska Gateway School District      200050
Lowe

In [54]:
df.loc[(df.fips.isnull()), 'school_district'] = (
    ak_district_lookup.reindex(
        df.loc[(df.fips.isnull()), 'County']
        .str.replace('Regional Educational Attendance Area', 'School District')
        .str.replace('Yukon Koyukuk', 'Yukon-Koyukuk')
        .str.replace('Pribilof Island', 'Pribilof'))
    .values)

In [55]:
df[df.fips.isnull() & df.school_district.isnull()]

,Disaster Number,Year,Region,Region Number,State Abbreviation,State,County,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,Disaster Close Out Date,fips,state_fips,county_fips,school_district


# Output data with assigned region numbers

In [56]:
df.to_csv('DataVizDisasterSummariesFV12.19.2016.with_region_codes.csv')